Data attributes and methods are collectively known as attributes in Python: a method is just an attribute that is callable. Besides data attributes and methods, we can also create properties, which can be used to replace a public data attribute with accessor methods (i.e., getter/setter), without changing the class interface. This agrees with the Uniform access principle:

> All services offered by a module should be available through a uniform notation, which does not betray whether they are implemented through storage or through computation.

In [1]:
import os
import json
import warnings
from urllib.request import urlopen

In [2]:
URL = 'https://raw.githubusercontent.com/AllenDowney/fluent-python-notebooks/master/19-dyn-attr-prop/oscon/data/osconfeed.json'  
JSON = 'osconfeed.json'

In [3]:
def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg)
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())
    
    with open(JSON) as fp:
        return json.load(fp)

In [4]:
feed = load()

In [5]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [6]:
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
484 events
357 speakers
 53 venues


In [7]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

In [8]:
feed['Schedule']['speakers'][-1]['serial']

141590

In [9]:
feed['Schedule']['events'][40]['name']

'There *Will* Be Bugs'

In [10]:
feed['Schedule']['events'][40]['speakers']

[3471, 5199]

In [11]:
from collections import abc

In [12]:
class FrozenJSON:
    """
    A read-only facade for navigating a JSON-like object using attribute notation.
    """
    
    def __init__(self, mapping):
        self.__data = dict(mapping)
        # Build a dict from the mapping argument. This serves two purposes:
        # ensures we got a dict (or something that can be converted to one)
        # and makes a copy for safety.
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

*It’s essential to recall that the `__getattr__` special method is only invoked by the interpreter when the usual process fails to retrieve an attribute (i.e., when the named attribute cannot be found in the instance, nor in the class or in its superclasses)*.

As shown, the FrozenJSON class has only two methods (`__init__`, `__getattr__`) and a `__data` instance attribute, so attempts to retrieve an attribute by any other name will trigger `__getattr__`. This method will first look if the `self.__data` dict has an attribute (**not a key!**) by that name; this allows FrozenJSON instances to handle any dict method such as items, by delegating to `self.__data.items()`. If `self.___data` doesn’t have an attribute with the given name, `__getattr__` uses name as a key to retrieve an item from `self.__dict`, and passes that item to `FrozenJSON.build`. This allows navigating through nested structures in the JSON data, as each nested mapping is converted to another FrozenJSON instance by the build class method.

In [13]:
raw_feed = load()
feed = FrozenJSON(raw_feed)
len(feed.Schedule.speakers)

357

In [14]:
sorted(feed.Schedule.keys())

['conferences', 'events', 'speakers', 'venues']

In [15]:
for key, value in sorted(feed.Schedule.items()): 
    print('{:3} {}'.format(len(value), key))

  1 conferences
484 events
357 speakers
 53 venues


In [16]:
feed.Schedule.speakers[40].name

'Tim Bray'

In [17]:
type(feed.Schedule.events[40])

__main__.FrozenJSON

In [18]:
talk = feed.Schedule.events[40]
talk.name

'There *Will* Be Bugs'

In [19]:
talk.speakers

[3471, 5199]

### The Invalid Attribute Name Problem

In [20]:
grad = FrozenJSON({'name': 'Jim Bo', 'class': 1982})

In [21]:
# You won’t be able to read grad.class because class is a reserved word in Python:
# grad.class
# It will give SyntaxError

In [22]:
# You can always do this, of course:
getattr(grad, 'class')

1982

But the idea of FrozenJSON is to provide convenient access to the data, so a better solution is checking whether a key in the mapping given to `FrozenJSON.__init__` is a keyword, and if so, append an _ to it.

A similar problem may arise if a key in the JSON is not a valid Python identifier:

` x = FrozenJSON({'2be':'or not'}) `

` x.2be `  ***SyntaxError***

Such problematic keys are easy to detect in Python 3 because the str class provides the `s.isidentifier()` method, which tells you whether s is a valid Python identifier according to the language grammar. But turning a key that is not a valid identifier into valid attribute name is not trivial. Two simple solutions would be raising an exception or replacing the invalid keys with generic names like attr_0, attr_1, and so on. 

In [23]:
import keyword

In [24]:
class FrozenJSON:
    """
    A read-only facade for navigating a JSON-like object using attribute notation.
    """
    
    def __init__(self, mapping):
        def __init__(self, mapping):
            self.__data = {}
            for key, value in mapping.items():
                if not key.isidentifier():
                    raise NameError('Not a valid Python identifier')
                if keyword.iskeyword(key): 
                    key += '_'
                self.__data[key] = value
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        else:
            return obj

## Flexible Object Creation with `__new__`
The special method that actually constructs an instance is `__new__`: it’s a class method (but gets special treatment, so the `@classmethod` decorator is not used), and it must return an instance.

The path just described, from `__new__` to `__init__`, is the most common, but not the only one. The `__new__` method can also return an instance of a different class, and when that happens, the interpreter does not call `__init__`.

In [25]:
def object_maker(the_class, some_arg):
    new_object = the_class.__new__(some_arg)
    if isinstance(new_object, the_class):
        the_class.__init__(new_object, some_arg)
    return new_object

In [26]:
class FrozenJSON:
    """
    Using new instead of build to construct new objects that may or may not
    be instances of FrozenJSON.
    """
    
    def __new__(cls, arg): 
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls) 
        elif isinstance(arg, abc.MutableSequence): 
            return [cls(item) for item in arg]
        else:
            return arg
    
    def __init__(self, mapping):
        def __init__(self, mapping):
            self.__data = {}
            for key, value in mapping.items():
                if not key.isidentifier():
                    raise NameError('Not a valid Python identifier')
                if keyword.iskeyword(key): 
                    key += '_'
                self.__data[key] = value
    
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON.build(self.__data[name])

The `__new__` method gets the class as the first argument because, usually, the created object will be an instance of that class.

### Restructuring the OSCON Feed with shelve

In [27]:
DB_NAME = 'schedule1_db'
CONFERENCE = 'conference.115'

In [28]:
class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)  # <2>

The `Record.__init__` method illustrates a popular Python hack. Recall that the `__dict__` of an object is where its attributes are kept—unless `__slots__` is declared in the class. So, updating an instance `__dict__` with a mapping is a quick way to create a bunch of attributes in that instance.

In [29]:
def load_db(db):
    raw_data = load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():  # <4>
        record_type = collection[:-1]  # <5>
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])  # <6>
            record['serial'] = key  # <7>
            db[key] = Record(**record)  # <8>

In [30]:
import shelve

In [31]:
db = shelve.open(DB_NAME)
if CONFERENCE not in db:
    # A quick way to determine if the database is populated is to look for a known key,
    # in this case conference.115—the key to the single conference record.
    # If the database is empty, call load_db(db) to load it
    load_db(db)

In [32]:
speaker = db['speaker.3471']

In [33]:
type(speaker)

__main__.Record

In [34]:
speaker.name, speaker.twitter

('Anna Martelli Ravenscroft', 'annaraven')

In [35]:
db.close()

### Linked Record Retrieval with Properties

In [36]:
DB_NAME = 'schedule2_db'
CONFERENCE = 'conference.115'

In [37]:
class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __eq__(self, other):  # <3>
        if isinstance(other, Record):
            return self.__dict__ == other.__dict__
        else:
            return NotImplemented

In [38]:
class MissingDatabaseError(RuntimeError):
    """Raised when a database is required but was not set."""  # <1>

In [39]:
class DbRecord(Record):  # <2>

    __db = None  # <3>

    @staticmethod  # <4>
    def set_db(db):
        DbRecord.__db = db  # <5>

    @staticmethod  # <6>
    def get_db():
        # I did not use a property to manage __db because of a crucial fact:
        # properties are class attributes designed to manage instance attributes.
        return DbRecord.__db

    @classmethod  # <7>
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]  # <8>
        except TypeError:
            if db is None:  # <9>
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:  # <10>
                raise

    def __repr__(self):
        if hasattr(self, 'serial'):  # <11>
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()  # <12>

In [40]:
class Event(DbRecord):  # <1>

    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key)  # <2>

    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'):  # <3>
            spkr_serials = self.__dict__['speakers']  # <4>
            fetch = self.__class__.fetch  # <5>
            self._speaker_objs = [fetch('speaker.{}'.format(key))
                                  for key in spkr_serials]  # <6>
        return self._speaker_objs  # <7>

    def __repr__(self):
        if hasattr(self, 'name'):  # <8>
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()  # <9>

2. The venue property builds a key from the venue_serial attribute, and passes it to the fetch class method, inherited from DbRecord.
3. The speakers property checks if the record has a _speaker_objs attribute.
4. If it doesn’t, the 'speakers' attribute is retrieved directly from the instance `__dict__` to avoid an infinite recursion, because the public name of this property is also speakers.
5. Get a reference to the fetch class method.
6. self._speaker_objs is loaded with a list of speaker records, using fetch.

In the venue property, the last line returns `self.__class__.fetch(key)`. Why not write that simply as `self.fetch(key)`? The simpler formula works with the specific dataset of the OSCON feed because there is no event record with a 'fetch' key. If even a single event record had a key named 'fetch', then within that specific Event instance, the reference self.fetch would retrieve the value of that field, instead of the fetch class method that Event  inherits from DbRecord. This is a subtle bug, and it could easily sneak through testing and blow up only in production when the venue or speaker records linked to that specific Event record are retrieved.

In [41]:
def load_db(db):
    raw_data = osconfeed.load()
    warnings.warn('loading ' + DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]  # <1>
        cls_name = record_type.capitalize()  # <2>
        cls = globals().get(cls_name, DbRecord)  # <3>
        if inspect.isclass(cls) and issubclass(cls, DbRecord):  # <4>
            factory = cls  # <5>
        else:
            factory = DbRecord  # <6>
        for record in rec_list:  # <7>
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record)  # <8>

## Using a Property for Attribute Validation

### LineItem Take #1: Class for an Item in an Order

In [42]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [43]:
raisins = LineItem('Golden raisins', 10, 6.95)

In [44]:
raisins.subtotal()

69.5

In [45]:
raisins.weight = -20 # garbage in...

In [46]:
raisins.subtotal() # garbage out...

-139.0

### LineItem Take #2: A Validating Property

In [47]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
    @property
    def weight(self):
        return self.__weight
    
    @weight.setter
    def weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('Weight cannot be negative.')

In [48]:
try:
    walnuts = LineItem('walnuts', 0, 10.00)
except ValueError:
    pass

## A Proper Look at Properties

Although often used as a decorator, the property built-in is actually a class. In Python, functions and classes are often interchangeable, because both are callable and there is no new operator for object instantiation, so invoking a constructor is no different than invoking a factory function. And both can be used as decorators, as long as they return a new callable that is a suitable replacement of the decorated function.

This is the full signature of the property constructor:

`property(fget=None, fset=None, fdel=None, doc=None)`

All arguments are optional, and if a function is not provided for one of them, the corresponding operation is not allowed by the resulting property object.

In [49]:
class LineItem:
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price
    
    def get_weight(self):
        return self.__weight
    
    def set_weight(self, value):
        if value > 0:
            self.__weight = value
        else:
            raise ValueError('Weight cannot be negative.')
    
    weight = property(get_weight, set_weight)

The classic form is better than the decorator syntax in some situations; the code of the property factory we’ll discuss shortly is one example. On the other hand, in a class body with many methods, the decorators make it explicit which are the getters and setters, without depending on the convention of using get and set prefixes in their names.

### Properties Override Instance Attributes
>**Properties are always class attributes, but they actually manage attribute access in the instances of the class**.

When an instance and its class both have a data attribute by the same name, the instance attribute overrides, or shadows, the class attribute—at least when read through that instance.

In [50]:
class Class:
    data = 'class attr'
    
    @property
    def prop(self):
        return 'prop value'

In [51]:
obj = Class()
vars(obj) 
# vars returns the __dict__ of obj, showing it has no instance attributes.

{}

In [52]:
obj.data

'class attr'

In [53]:
obj.data = 'new instance attr'
# Writing to obj.data creates an instance attribute

In [54]:
obj.data
# Now reading from obj.data retrieves the value of the instance attribute. When
# read from the obj instance, the instance data shadows the class data.

'new instance attr'

In [55]:
vars(obj)

{'data': 'new instance attr'}

In [56]:
Class.data # The Class.data attribute is intact

'class attr'

Now, let’s try to override the prop attribute on the obj instance.

In [57]:
Class.prop
# Reading prop directly from Class retrieves the property object itself, without
# running its getter method

In [58]:
obj.prop
# Reading obj.prop executes the property getter

'prop value'

In [59]:
try:
    obj.prop = 'foo'
except AttributeError as err:
    print(str(err))
# Trying to set an instance prop attribute fails.

can't set attribute


In [60]:
obj.__dict__['prop'] = 'foo'
# Putting 'prop' directly in the obj.__dict__ works

In [61]:
vars(obj)
# We can see that obj now has two instance attributes: attr and prop

{'data': 'new instance attr', 'prop': 'foo'}

In [62]:
obj.prop
# However, reading obj.prop still runs the property getter. The property is not
# shadowed by an instance attribute. __getattr__ is not triggered since prop already exists.

'prop value'

In [63]:
Class.prop = 'property destroyed!'
# Overwriting Class.prop destroys the property object

In [64]:
obj.prop

'foo'

In [65]:
Class.prop

'property destroyed!'

As a final demonstration, we’ll add a new property to Class, and see it overriding an instance attribute.

In [66]:
obj.data

'new instance attr'

In [67]:
Class.data

'class attr'

In [68]:
Class.data = property(lambda self: 'the "data" prop value')
# Overwrite Class.data with a new property

In [69]:
obj.data
# obj.data is now shadowed by the Class.data property

'the "data" prop value'

In [70]:
del Class.data

In [71]:
obj.data

'new instance attr'

The main point of this section is that an expression like `obj.attr` does not search for `attr` starting with `obj`. **The search actually starts at `obj.__class__`, and only if there is no *`property`* named `attr` in the class, Python looks in the `obj` instance itself and if there is no object named `attr`, it searches in the class for an attribute named `attr`**. This rule applies not only to properties but to a whole category of descriptors, the *overriding descriptors*.

### Property Documentation

> `weight = property(get_weight, set_weight, doc='weight in kilograms')`

When property is deployed as a decorator, the docstring of the getter method—the one with the @property decorator itself—is used as the documentation of the property as a whole.

```
class Foo:

    @property
    def bar(self):
        '''The bar attribute'''
        return self.__dict__['bar']

    @bar.setter
    def bar(self, value):
        self.__dict__['bar'] = value
```

## Coding a Property Factory

In [72]:
def quantity(storage_name):
    
    def qty_getter(instance):
        return instance.__dict__[storage_name]
    
    def qty_setter(instance, value):
        if value > 0:
            instance.__dict__[storage_name] = value

In [73]:
class LineItem:
    weight = quantity('weight') 
    price = quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight 
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price 

Recall that properties are class attributes. When building each quantity property, we need to pass the name of the LineItem attribute that will be managed by that specific property.

In [74]:
nutmeg = LineItem('Moluccan nutmeg', 8, 13.95)
nutmeg.weight, nutmeg.price

(8, 13.95)

In [75]:
sorted(vars(nutmeg).items())

[('description', 'Moluccan nutmeg'), ('price', 13.95), ('weight', 8)]

### Handling Attribute Deletion

In [76]:
class BlackKnight:
    def __init__(self):
        self.members = ['an arm', 'another arm',
                        'a leg', 'another leg']
        self.phrases = ["'Tis but a scratch.",
                        "It's just a flesh wound.",
                        "I'm invincible!",
                        "All right, we'll call it a draw."]
    
    @property
    def member(self):
        print('next member is:')
        return self.members[0]
    
    @member.deleter
    def member(self):
        text = 'BLACK KNIGHT (loses {})\n-- {}'
        print(text.format(self.members.pop(0), self.phrases.pop(0)))

In [77]:
knight = BlackKnight()
knight.member

next member is:


'an arm'

In [78]:
del knight.member

BLACK KNIGHT (loses an arm)
-- 'Tis but a scratch.


In [79]:
del knight.member

BLACK KNIGHT (loses another arm)
-- It's just a flesh wound.


In [80]:
del knight.member

BLACK KNIGHT (loses a leg)
-- I'm invincible!


In [81]:
del knight.member

BLACK KNIGHT (loses another leg)
-- All right, we'll call it a draw.


> `member = property(member_getter, fdel=member_deleter)`

If you are not using a property, attribute deletion can also be handled by implementing the lower-level `__delattr__` special method